In [3]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Retrieve API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    print("API key not loaded. Please check your .env file.")
else:
    print(f"API key loaded: {api_key[:5]}...")  # Partial display for security


API key loaded: sk-pr...


In [4]:
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

try:
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "Say hello!"}]
    )
    print(response)
except Exception as e:
    print("Error during API call:", e)

ChatCompletion(id='chatcmpl-AlEeQU05JCmwaWSyufkMBerKLlqg5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1735821098, model='gpt-4-0613', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=10, total_tokens=20, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [ ]:
import json
import os
import time
from datetime import datetime
from typing import List, Dict, Set
from openai import OpenAI
from rich.console import Console
from rich.progress import Progress, SpinnerColumn, TimeElapsedColumn
import asyncio
import aiohttp
from concurrent.futures import ThreadPoolExecutor
from functools import partial

# Define available styles with accessibility-focused descriptions
STYLES = {
    "Cartoon": "Simple forms with clear outlines and flat colors, ideal for cognitive processing and bounding box annotation",
    "Realistic": "Natural representations with clear object separation and recognizable elements for real-world connection",
    "Artistic": "Stylized interpretation with distinct elements and good spacing for visual clarity",
    "Minimalistic": "Essential elements only with high contrast and clear object boundaries for reduced cognitive load",
    "Digital Art": "Clean rendering with sharp edges and distinct object separation for clear visual hierarchy",
    "3D Rendered": "Moderate depth with clear object boundaries and minimal overlap for spatial understanding",
    "Geometric": "Basic shapes with clear spacing and easy-to-mark boundaries for simplified processing",
    "Retro": "Simplified vintage style with bold shapes and clear figure-ground separation for visual distinction",
    "Storybook": "Simple but engaging style with clear object definition and comfortable viewing for enhanced comprehension",
    "Technical": "Precise linework with systematic layout and well-defined components for structured understanding"
}

# Define templates with clear accessibility focus
TEMPLATES = {
    "Basic Object Focus": {
        "description": "Single-plane arrangement focusing on key objects",
        "requirements": [
            "Place 3–5 objects against a simple, neutral background",
            "Maintain maximum spacing between objects for clear annotation",
            "Use consistent size and emphasis for all objects—no focal object",
            "Ensure high contrast between objects and background",
            "Do not imply any scene, narrative, or grouping",
            "Avoid visual hierarchy or object alignment"
        ],
        "cognitive_benefits": "Reduces cognitive load through clear, isolated presentation"
    },
    "Contextual Scene": {
        "description": "Horizontal scene with fixed viewpoint and clear object relationships",
        "requirements": [
            "Create single-perspective scene with no depth variation",
            "Place all objects on the same horizontal plane",
            "Maintain equal size for all objects to avoid depth illusion",
            "Use simple backdrop with no more than one environmental element",
            "Keep at least 20% spacing between each object",
            "Objects must be arranged in a single line horizontally"
        ],
        "cognitive_benefits": "Supports understanding through simple, consistent spatial relationships"
    },
    "Educational Layout": {
        "description": "Progressive sequence showing clear step-by-step relationship",
        "requirements": [
            "Organize objects in strict left-to-right sequence only",
            "Each object must be connected to the next with a clear visual indicator (arrow or line)",
            "Objects must decrease in size from left to right by 10-15%",
            "Include numbered positions (1,2,3) near each object",
            "Use consistent spacing that narrows between each subsequent object",
            "Maximum of 4 objects to maintain clear progression"
        ],
        "cognitive_benefits": "Facilitates sequential processing and cause-effect understanding"
    },
    "Multi-Level Detail": {
        "description": "Layered arrangement with distinct depth zones",
        "requirements": [
            "Create exactly three depth layers with clear separation",
            "Foreground must be 2x larger than midground",
            "Midground must be 2x larger than background",
            "Use different heights for each layer",
            "No horizontal alignment between layers",
            "Each layer must have distinct lightness value"
        ],
        "cognitive_benefits": "Helps process information hierarchy through clear spatial organization"
    },
    "Grid Layout": {
        "description": "Systematic grid-based arrangement for maximum visual clarity",
        "requirements": [
            "Place objects in a fixed 2x2 or 3x3 grid structure",
            "Use equal size cells with thick borders",
            "Maintain identical size for all objects",
            "Keep minimum 25% margin within each cell",
            "Objects must be centered in their cells",
            "No diagonal or complex arrangements"
        ],
        "cognitive_benefits": "Supports systematic visual processing through clear, structured organization"
    }
}


# System prompt with key requirements
SYSTEM_PROMPT = """You are an expert at creating accessible image generation prompts for a research project on cognitive accessibility.

RESEARCH CONTEXT:
- Target Audience: People with cognitive disabilities and reading difficulties
- Primary Goal: Enhance text comprehension through accessible multimodal content
- Research Setting: Master's thesis investigating multimodal accessibility
- Validation Process: Images will be annotated by students using Label Studio

MANDATORY REQUIREMENTS FOR ALL PROMPTS:
1. Objects and Layout:
   - Include EXACTLY 3-5 distinct physical objects
   - Maintain clear spacing between objects
   - No overlapping elements
   - Clear boundaries between objects

2. Content Restrictions:
   - NO text or writing elements
   - NO dynamic actions or motion
   - NO abstract concepts
   - NO complex backgrounds"""

class PromptGenerator:
    def __init__(self, api_key: str):
        self.console = Console()
        self.client = OpenAI(api_key=api_key)
        self.output_path = os.path.join('..', 'output_files', 'generated_prompts.json')
        
    async def generate_prompt_for_template(self, text: str, template_name: str, template_info: dict) -> Dict:
        """Generate prompt for a single template asynchronously."""
        try:
            user_prompt = f'''Given this simplified text: "{text}"

Create a prompt for the {template_name} template.

Template Description: {template_info['description']}

Template Requirements:
{chr(10).join(f"- {req}" for req in template_info['requirements'])}

Available styles: {", ".join(STYLES.keys())}

Format as:
style: [pick one style]
prompt: [your prompt]'''

            response = await asyncio.get_event_loop().run_in_executor(
                None,
                partial(
                    self.client.chat.completions.create,
                    model="gpt-4",
                    messages=[
                        {"role": "system", "content": SYSTEM_PROMPT},
                        {"role": "user", "content": user_prompt}
                    ],
                    temperature=0.7
                )
            )
            
            content = response.choices[0].message.content.strip()
            style = content.split('style:', 1)[1].split('\n', 1)[0].strip()
            prompt = content.split('prompt:', 1)[1].strip()
            
            if style in STYLES:
                return {
                    'template_name': template_name,
                    'style': style,
                    'prompt': prompt
                }
                
        except Exception as e:
            self.console.print(f"[yellow]Error with template {template_name}: {str(e)}")
            return None
            
    async def process_single_sample(self, sample: Dict, index: int) -> Dict:
        """Process a single sample with all templates."""
        tasks = []
        for template_name, template_info in TEMPLATES.items():
            tasks.append(self.generate_prompt_for_template(
                sample['simplified'], 
                template_name, 
                template_info
            ))
            
        prompts = await asyncio.gather(*tasks)
        prompts = [p for p in prompts if p is not None]
        
        return {
            'index': index,
            'id': sample['id'],
            'simplified_text': sample['simplified'],
            'template_prompts': prompts
        }

    def save_results(self, results: List[Dict]):
        """Save results to file."""
        os.makedirs(os.path.dirname(self.output_path), exist_ok=True)
        with open(self.output_path, 'w') as f:
            json.dump(results, f, indent=2)

    async def process_all_samples(self):
        """Process all samples at once."""
        input_path = os.path.join('..', 'output_files', 'complete_dataset.json')
        
        try:
            # Load samples
            samples = []
            with open(input_path, 'r') as f:
                for line in f:
                    samples.append(json.loads(line.strip()))
            
            # Take first 100 samples
            target_samples = samples[:100]
            
            all_results = []
            
            # Process all samples with progress bar
            with Progress(
                SpinnerColumn(),
                *Progress.get_default_columns(),
                TimeElapsedColumn(),
                console=self.console
            ) as progress:
                task = progress.add_task("Processing samples...", total=len(target_samples))
                
                # Process samples concurrently with rate limiting
                semaphore = asyncio.Semaphore(5)  # Limit concurrent API calls
                
                async def process_with_rate_limit(sample, index):
                    async with semaphore:
                        return await self.process_single_sample(sample, index)
                
                tasks = [process_with_rate_limit(sample, i) 
                        for i, sample in enumerate(target_samples)]
                
                for coro in asyncio.as_completed(tasks):
                    result = await coro
                    all_results.append(result)
                    progress.update(task, advance=1)
                    
                    # Save intermediate results every 10 samples
                    if len(all_results) % 10 == 0:
                        self.save_results(all_results)
            
            # Final save
            self.save_results(all_results)
            
            self.console.print(f"\n[green]Successfully processed all {len(all_results)} samples!")
            
        except Exception as e:
            self.console.print(f"[red]Error: {str(e)}")

def main():
    # Get API key
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("OPENAI_API_KEY environment variable not set")
    
    # Initialize generator
    generator = PromptGenerator(api_key)
    
    try:
        # If we're in IPython/Jupyter, use existing event loop
        loop = asyncio.get_event_loop()
        if loop.is_running():
            # We're in a notebook with a running event loop
            asyncio.ensure_future(generator.process_all_samples())
        else:
            # We're either in a script or notebook without running loop
            loop.run_until_complete(generator.process_all_samples())
    except RuntimeError:
        # If no event loop exists, create and manage a new one
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        loop.run_until_complete(generator.process_all_samples())

if __name__ == "__main__":
    main()

Successfully processed all 100 samples!